# Download All Stock Prices

We get historical stock prices from [tiingo](https://www.tiingo.com/).          
You need to go to ["pricing"](https://www.tiingo.com/account/billing/pricing) and update your account to "Power" (this costs $10 per month and you can cancel after one month if you don't need any additional new data).

In [1]:
import pandas as pd
import requests, zipfile, io
import os
from pathlib import Path

from tiingo import TiingoClient                       
tiingo = TiingoClient({'api_key':'f10f5d79cf2567b045cc7e9b595a6fec0f5321d2'})     # Put your API key here

apiKey = 'f10f5d79cf2567b045cc7e9b595a6fec0f5321d2'                               # Put your API key here

Get the ticker symbols:

In [2]:
# SEC CIKs:
sec = pd.read_json('https://www.sec.gov/files/company_tickers.json').transpose()
sec = sec.rename(columns={'cik_str':'cik'})

# Tiingo ticker symbols:
r = requests.get('https://apimedia.tiingo.com/docs/tiingo/daily/supported_tickers.zip')
z = zipfile.ZipFile(io.BytesIO(r.content))
z.namelist() 
filename = z.namelist()[0]
tngo = pd.read_csv(z.open(filename))

# Merge the tables:
all_shares = sec.merge(tngo, on='ticker', how='outer')
all_shares = all_shares[all_shares.assetType!='ETF']         # Drop ETFs

# Save the table:
symbols = all_shares.sort_values(['cik','startDate']).groupby('cik',as_index=False).first()
Path('data/ticker_symbols').mkdir(parents=True, exist_ok=True) 
symbols.to_csv('data/ticker_symbols/symbols.csv', index=False)

Read the symbols table:

In [3]:
symbols = pd.read_csv('data/ticker_symbols/symbols.csv',index_col=0)
symbols

,ticker,title,exchange,assetType,priceCurrency,startDate,endDate
cik,,,,,,,
1750,AIR,AAR CORP,NYSE,Stock,USD,1984-07-19,2021-04-16
1800,ABT,ABBOTT LABORATORIES,NYSE,Stock,USD,1983-04-06,2021-04-16
1961,WDDD,WORLDS INC,OTCQB,Stock,USD,1998-10-20,2021-04-16
2098,ACU,ACME UNITED CORP,NYSE MKT,Stock,USD,1984-09-07,2021-04-16
2178,AE,"ADAMS RESOURCES & ENERGY, INC.",NYSE MKT,Stock,USD,1984-09-07,2021-04-16
...,...,...,...,...,...,...,...
1852736,TRIS,Tristar Acquisition I Corp.,NASDAQ,Stock,USD,NaN,NaN
1852931,NHLDW,NewHold Investment Corp. II,NASDAQ,Stock,USD,2017-01-23,2021-02-25
1853404,CEAS,CEA Space Partners I Corp.,NaN,NaN,NaN,NaN,NaN


Use SPY to get all trading days:

In [4]:
trading_days = pd.to_datetime( tiingo.get_dataframe('SPY','2009-04-15').index ).tz_convert(None)
trading_days

DatetimeIndex(['2009-04-15', '2009-04-16', '2009-04-17', '2009-04-20',
               '2009-04-21', '2009-04-22', '2009-04-23', '2009-04-24',
               '2009-04-27', '2009-04-28',
               ...
               '2021-04-05', '2021-04-06', '2021-04-07', '2021-04-08',
               '2021-04-09', '2021-04-12', '2021-04-13', '2021-04-14',
               '2021-04-15', '2021-04-16'],
              dtype='datetime64[ns]', name='date', length=3023, freq=None)

Download all stock prices (this might take about 1 hour):

In [5]:
startDate = '2009-01-01'

table_names = ['close','high','low','open','volume','adjClose','adjHigh','adjLow','adjOpen','adjVolume','divCash','splitFactor']

results = {t:pd.DataFrame(index=trading_days) for t in table_names}   # The dictionary "results" contains all the tables

In [6]:
from IPython.display import display, clear_output

for i,ticker in enumerate(symbols.ticker):
    
    clear_output(wait=True)
    display(ticker, str(round(i/len(symbols),4)*100) + '% finished')
    
    response = requests.get('https://api.tiingo.com/tiingo/daily/'+ticker+'/prices?startDate='+startDate+'&token='+apiKey)

    if response.ok & (len(response.json())>0):                     # If tiingo has this asset
        data = pd.DataFrame(response.json()).set_index('date')     # Put the response into  a table
        data.index = pd.to_datetime(data.index).tz_convert(None)
        for t in results.keys():    
            results[t][ticker] = data[t]                           # Add data as new column into each table.

        


results['close']        # Show table with all close prices

'VRAR'

'99.99% finished'

,AIR,ABT,WDDD,ACU,AE,BKTI,AMD,AEM,APD,AKZOF,...,KVSB,TIOAU,MSDAU,GGPIU,NECB,HITIF,AYRWF,OUTR,ABCO,NHLDW
date,,,,,,,,,,,,,,,,,,,,,
2009-04-15,14.43,42.66,0.1700,7.4300,14.65,0.6400,3.44,49.32,59.79,41.700,...,NaN,NaN,NaN,NaN,7.15,NaN,NaN,NaN,18.29,NaN
2009-04-16,15.04,42.69,0.1700,7.6000,14.84,0.6300,3.57,45.73,60.48,44.400,...,NaN,NaN,NaN,NaN,7.15,NaN,NaN,NaN,18.29,NaN
2009-04-17,15.17,43.89,0.1700,7.2000,15.00,0.5500,3.56,43.65,60.65,44.400,...,NaN,NaN,NaN,NaN,7.30,NaN,NaN,NaN,18.52,NaN
2009-04-20,14.01,44.09,0.1700,7.3800,14.50,0.5598,3.31,46.97,57.79,44.400,...,NaN,NaN,NaN,NaN,7.30,NaN,NaN,NaN,17.39,NaN
2009-04-21,14.66,44.37,0.1700,7.3900,14.50,0.5600,3.36,45.53,58.05,44.400,...,NaN,NaN,NaN,NaN,7.10,NaN,NaN,NaN,17.06,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-12,40.92,121.04,0.3925,38.6900,26.00,4.6400,78.58,60.12,282.88,116.300,...,10.27,10.0000,10.230,10.000,16.45,0.550100,27.8700,NaN,NaN,NaN
2021-04-13,40.23,123.01,0.3850,38.7501,26.34,4.5100,80.19,61.25,285.11,117.100,...,10.20,10.0475,10.345,10.005,16.45,0.526000,27.9546,NaN,NaN,NaN
2021-04-14,41.08,121.50,0.3900,38.3057,26.59,4.6800,78.55,60.80,282.88,117.100,...,10.24,10.0200,10.460,10.010,16.45,0.503515,28.0400,NaN,NaN,NaN


Save the stock price tables:

In [7]:
Path('data/tiingo').mkdir(parents=True, exist_ok=True)   # Generate directory.

for t in results.keys():
    results[t].to_csv('data/tiingo/'+t+'.csv')

And now you can read the files like this:

In [8]:
adjClose = pd.read_csv('data/tiingo/adjClose.csv', index_col='date', parse_dates=['date'])
adjClose

,AIR,ABT,WDDD,ACU,AE,BKTI,AMD,AEM,APD,AKZOF,...,KVSB,TIOAU,MSDAU,GGPIU,NECB,HITIF,AYRWF,OUTR,ABCO,NHLDW
date,,,,,,,,,,,,,,,,,,,,,
2009-04-15,12.989114,15.438419,0.1700,5.719596,11.176977,0.542737,3.44,42.306611,44.210609,32.673910,...,NaN,NaN,NaN,NaN,5.931042,NaN,NaN,NaN,9.145,NaN
2009-04-16,13.538203,15.449275,0.1700,5.850461,11.321935,0.534257,3.57,39.227115,44.720816,34.789487,...,NaN,NaN,NaN,NaN,5.931042,NaN,NaN,NaN,9.145,NaN
2009-04-17,13.655222,15.883549,0.1700,5.542542,11.444004,0.466415,3.56,37.442894,44.846520,34.789487,...,NaN,NaN,NaN,NaN,6.055469,NaN,NaN,NaN,9.260,NaN
2009-04-20,12.611053,15.955928,0.1700,5.681106,11.062537,0.474725,3.31,40.290785,42.731746,34.789487,...,NaN,NaN,NaN,NaN,6.055469,NaN,NaN,NaN,8.695,NaN
2009-04-21,13.196148,16.057258,0.1700,5.688804,11.062537,0.474895,3.36,39.055555,42.923998,34.789487,...,NaN,NaN,NaN,NaN,5.889566,NaN,NaN,NaN,8.530,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-04-12,40.920000,120.593358,0.3925,38.690000,26.000000,4.640000,78.58,60.120000,282.880000,116.300000,...,10.27,10.0000,10.230,10.000,16.450000,0.550100,27.8700,NaN,NaN,NaN
2021-04-13,40.230000,122.556089,0.3850,38.750100,26.340000,4.510000,80.19,61.250000,285.110000,117.100000,...,10.20,10.0475,10.345,10.005,16.450000,0.526000,27.9546,NaN,NaN,NaN
2021-04-14,41.080000,121.500000,0.3900,38.305700,26.590000,4.680000,78.55,60.800000,282.880000,117.100000,...,10.24,10.0200,10.460,10.010,16.450000,0.503515,28.0400,NaN,NaN,NaN
